In [1]:
!pip install torch transformers accelerate tqdm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm

# Cihaz kontrolü (GPU varsa GPU, yoksa CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz: {device}")

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\Yusuf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\Yusuf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Kullanılan cihaz: cuda


In [2]:
# --- BURAYI GÜNCELLE ---
model_yolu = r"C:\Users\Yusuf\.cache\kagglehub\models\kubikay\mathmodellarge\pyTorch\default\1\final_unwrapped"# -----------------------

print("Model yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(
    model_yolu,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
)

# Eğer tokenizer'ın pad_token'ı yoksa (Llama gibi modellerde gerekebilir)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Model yükleniyor...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 292/292 [00:00<00:00, 435.20it/s, Materializing param=transformer.wte.weight]             


In [3]:
def evaluate_model_performance(model, tokenizer, test_texts, batch_size=4):
    model.eval()
    total_top1 = 0
    total_top5 = 0
    total_tokens = 0
    total_loss = 0

    # Verileri batch'lere bölüyoruz
    for i in tqdm(range(0, len(test_texts), batch_size), desc="Değerlendiriliyor"):
        batch_texts = test_texts[i : i + batch_size]

        # Tokenization
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        labels = inputs["input_ids"].clone()

        with torch.no_grad():
            outputs = model(**inputs, labels=labels)
            logits = outputs.logits
            loss = outputs.loss

            # Kaydırma (Shift): Modelin N. tahmini N+1. token ile kıyaslanır
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()

            # Padding olan yerleri (ignore_index = -100 veya pad_id) hesaplamadan çıkarıyoruz
            mask = shift_labels != tokenizer.pad_token_id

            # Top-k hesaplama
            _, top5_indices = shift_logits.topk(5, dim=-1)

            # Doğru tahminleri say (Sadece maskelenmemiş yani gerçek tokenlar için)
            top1_correct = ((top5_indices[:, :, 0] == shift_labels) & mask).sum().item()
            top5_correct = ((top5_indices == shift_labels.unsqueeze(-1)).any(dim=-1) & mask).sum().item()

            total_top1 += top1_correct
            total_top5 += top5_correct
            total_tokens += mask.sum().item()
            total_loss += loss.item() * mask.sum().item()

    avg_top1 = (total_top1 / total_tokens) * 100
    avg_top5 = (total_top5 / total_tokens) * 100
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens)).item()

    return {
        "Top-1 Accuracy (%)": round(avg_top1, 2),
        "Top-5 Accuracy (%)": round(avg_top5, 2),
        "Perplexity": round(perplexity, 4)
    }

In [13]:
def evaluate_sft_performance(model, tokenizer, test_samples, batch_size=2):
    model.eval()
    total_top1, total_top5, total_tokens, total_loss = 0, 0, 0, 0
    
    # SFT modellerinde genellikle cevap bir ayırıcıdan sonra gelir
    # Kullandığın modele göre "Cevap:" veya "\n### Response:" gibi güncelleyebilirsin
    response_delimiter = "Cevap:" 

    for i in tqdm(range(0, len(test_samples), batch_size)):
        batch_texts = test_samples[i : i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        labels = inputs["input_ids"].clone()
        
        # --- SFT MASKELME ---
        # Soru kısmındaki tokenları başarı hesabına dahil etmemek için -100 ile maskeliyoruz
        for j, text in enumerate(batch_texts):
            if response_delimiter in text:
                # Soru ve cevap kısmını ayırıyoruz
                prompt_part = text.split(response_delimiter)[0] + response_delimiter
                prompt_token_len = len(tokenizer.encode(prompt_part, add_special_tokens=False))
                # Soru kısmını -100 yap (loss ve doğruluk hesabında görmezden gelinir)
                labels[j, :prompt_token_len] = -100

        with torch.no_grad():
            outputs = model(**inputs, labels=labels)
            shift_logits = outputs.logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()
            
            # Sadece -100 olmayan (yani cevap olan) yerleri al
            mask = shift_labels != -100
            
            if mask.sum() == 0: continue # Boş batch kontrolü

            _, top5_indices = shift_logits.topk(5, dim=-1)
            
            top1_correct = ((top5_indices[:, :, 0] == shift_labels) & mask).sum().item()
            top5_correct = ((top5_indices == shift_labels.unsqueeze(-1)).any(dim=-1) & mask).sum().item()
            
            total_top1 += top1_correct
            total_top5 += top5_correct
            total_tokens += mask.sum().item()
            total_loss += outputs.loss.item() * mask.sum().item()

    return {
        "SFT Top-1 Acc (%)": round((total_top1 / total_tokens) * 100, 2),
        "SFT Top-5 Acc (%)": round((total_top5 / total_tokens) * 100, 2),
        "SFT Perplexity": round(torch.exp(torch.tensor(total_loss / total_tokens)).item(), 4)
    }

In [5]:
from datasets import load_dataset

# 1. Veri setini 'main' config'i ile yüklüyoruz
print("GSM8K-TR (malhajar) veri seti yükleniyor...")
dataset = load_dataset("malhajar/gsm8k-tr", "main")

# 2. Test verisini hazırla
test_samples = []

# Bu veri setinde split isimleri genelde 'train' ve 'test'tir.
# Kontrol amaçlı split ismini belirliyoruz:
split_name = "test" if "test" in dataset else "train"

for example in dataset[split_name]:
    # Sütun isimleri bu veri setinde 'question' ve 'answer'dır.
    full_text = f"Soru: {example['question']}\nCevap: {example['answer']}"
    test_samples.append(full_text)

print(f"Toplam {len(test_samples)} örnek hazırlandı. ({split_name} spliti kullanılıyor)")

# 3. Değerlendirmeyi Çalıştır (Önceki hücredeki fonksiyonu kullanır)
print("\n--- GSM8K-TR Üzerinde Değerlendirme Başlatılıyor ---")
results = evaluate_model_performance(model, tokenizer, test_samples, batch_size=2)

print("\n--- GSM8K-TR EVAL SONUÇLARI ---")
for metrik, deger in results.items():
    print(f"{metrik}: {deger}")

GSM8K-TR (malhajar) veri seti yükleniyor...
Toplam 1318 örnek hazırlandı. (test spliti kullanılıyor)

--- GSM8K-TR Üzerinde Değerlendirme Başlatılıyor ---


Değerlendiriliyor: 100%|██████████| 659/659 [00:31<00:00, 21.21it/s]


--- GSM8K-TR EVAL SONUÇLARI ---
Top-1 Accuracy (%): 45.55
Top-5 Accuracy (%): 67.07
Perplexity: 15.9224


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# --- AYARLAR ---
base_model_yolu = r"C:\Users\Yusuf\.cache\kagglehub\models\kubikay\mathmodellarge\pyTorch\default\1\final_unwrapped"# -----------------------
lora_weights_yolu = r"C:\Users\Yusuf\.cache\kagglehub\models\kubikay\sft-lora\pyTorch\default\1\final_model"
# ---------------

print("Base model ve Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(base_model_yolu)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_yolu,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("LoRA ağırlıkları (PEFT) giydiriliyor...")
model = PeftModel.from_pretrained(base_model, lora_weights_yolu)
model = model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("LoRA modeli başarıyla birleştirildi ve hazır!")

Base model ve Tokenizer yükleniyor...


Loading weights: 100%|██████████| 292/292 [00:00<00:00, 403.10it/s, Materializing param=transformer.wte.weight]             


LoRA ağırlıkları (PEFT) giydiriliyor...
LoRA modeli başarıyla birleştirildi ve hazır!


In [14]:
from datasets import load_dataset

# 1. Veriyi çek
dataset = load_dataset("malhajar/gsm8k-tr", "main")
test_samples = [f"Soru: {ex['question']}\nCevap: {ex['answer']}" for ex in dataset["test"]]

# 2. Fonksiyonu Çalıştır
# 'evaluate_lora_performance' fonksiyonunun daha önceki hücrelerde tanımlı olduğundan emin ol!
print("\n--- LoRA Modeli GSM8K-TR Üzerinde Test Ediliyor ---")
results = evaluate_sft_performance(model, tokenizer, test_samples, batch_size=2)

# 3. Sonuçları Yazdır
print("\n--- ANALİZ SONUÇLARI ---")
for metrik, deger in results.items():
    print(f"{metrik}: {deger}")


--- LoRA Modeli GSM8K-TR Üzerinde Test Ediliyor ---


100%|██████████| 659/659 [00:58<00:00, 11.31it/s]


--- ANALİZ SONUÇLARI ---
SFT Top-1 Acc (%): 70.36
SFT Top-5 Acc (%): 85.13
SFT Perplexity: 4.3515


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# --- AYARLAR ---

lora_weights_yolu_2 = r"C:\Users\Yusuf\.cache\kagglehub\models\kubikay\sft-v2\pyTorch\default\1\final_model"
# ---------------

print("Base model ve Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(base_model_yolu)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_yolu,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("LoRA ağırlıkları (PEFT) giydiriliyor...")
model = PeftModel.from_pretrained(base_model, lora_weights_yolu_2)
model = model.to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("LoRA modeli başarıyla birleştirildi ve hazır!")

Base model ve Tokenizer yükleniyor...


Loading weights: 100%|██████████| 292/292 [00:00<00:00, 358.84it/s, Materializing param=transformer.wte.weight]             


LoRA ağırlıkları (PEFT) giydiriliyor...
LoRA modeli başarıyla birleştirildi ve hazır!


In [16]:
from datasets import load_dataset

# 1. Veri setini 'main' config'i ile yüklüyoruz
print("GSM8K-TR (malhajar) veri seti yükleniyor...")
dataset = load_dataset("malhajar/gsm8k-tr", "main")

# 2. Test verisini hazırla
test_samples = []

# Bu veri setinde split isimleri genelde 'train' ve 'test'tir.
# Kontrol amaçlı split ismini belirliyoruz:
split_name = "test" if "test" in dataset else "train"

for example in dataset[split_name]:
    # Sütun isimleri bu veri setinde 'question' ve 'answer'dır.
    full_text = f"Soru: {example['question']}\nCevap: {example['answer']}"
    test_samples.append(full_text)

print(f"Toplam {len(test_samples)} örnek hazırlandı. ({split_name} spliti kullanılıyor)")

# 3. Değerlendirmeyi Çalıştır (Önceki hücredeki fonksiyonu kullanır)
print("\n--- GSM8K-TR Üzerinde Değerlendirme Başlatılıyor ---")
results = evaluate_model_performance(model, tokenizer, test_samples, batch_size=2)

print("\n--- GSM8K-TR EVAL SONUÇLARI ---")
for metrik, deger in results.items():
    print(f"{metrik}: {deger}")

GSM8K-TR (malhajar) veri seti yükleniyor...
Toplam 1318 örnek hazırlandı. (test spliti kullanılıyor)

--- GSM8K-TR Üzerinde Değerlendirme Başlatılıyor ---


Değerlendiriliyor: 100%|██████████| 659/659 [00:59<00:00, 10.99it/s]


--- GSM8K-TR EVAL SONUÇLARI ---
Top-1 Accuracy (%): 54.83
Top-5 Accuracy (%): 74.18
Perplexity: 8.2816
